# Tutorial for extracting graph represenatations of all images

## Step 1: Register images from the same FOV

In [1]:
from numpy.random import shuffle
from pathlib import Path
from novas3d.registration import ImageRegistration
import pickle
from os import mkdir
from os.path import exists
from novas3d.utilities import get_mov_files

In [2]:
#load dictionary for matched volumes
with open('novas3d_example_data/matched_stacks_init.pickle', 'rb') as f:
    dic = pickle.load(f)
sorted(dic['67/XYZres387'])

FileNotFoundError: [Errno 2] No such file or directory: 'novas3d_example_data/matched_stacks_init.pickle'

In [ ]:
mouse_ids_path = Path('novas3d_example_data/')#each mouse has its own folder with raw data in it
mouse_ids = list(mouse_ids_path.glob('*?[0-9]/*res*?[0-9].tif'))#grab folder names/mouse ids
images = sorted([x.as_posix() for x in mouse_ids if '_0001' not in x.as_posix()])
images = [x for x in images if  any(y in x for y in list(dic.keys()))]

unused_keys = [x for x in list(dic.keys()) if not  any(x in y for y in images)]
print(len(images))
shuffle(images)
params = { 
    'out_directory': 'novas3d_example_data/raw_warped/', # path to output directory
    'in_filename_extension': '.tif', # initial filename extension of unregistered images
    'final_filename_extension': '_warped.tif', # modified filename extension following registration
    'timepoint_suffixes': ['_0001'], # list of all timepoint suffixes present ie _0001, _0002, _0003, ...
    'sigma': 2, # Smoothing sigma applied to iamges prior to registration, used by ANTs rigid registration
    'flip': True, # flip images based on direction of aquision to match the reference. ONly works for files that were origionally olympus oir files
    'dic': dic # dictionary of matched aqusitions
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])


*class ImageRegistration(images, out_directory, in_filename_extension, final_filename_extension, timepoint_suffixes, sigma, flip, dic, skip_finished=False)*

Register images using ANTs registration.

**Args:**
- **images (list):** List of image files to register. Each time point should be its own file. Files should be tif stacks. Channel 1 should be the colour channel.
- **out_directory (str):** Output directory to save the registered images.
- **in_filename_extension (str):** Initial filename extension of the unregistered images.
- **final_filename_extension (str):** Final filename extension of the registered images.
- **timepoint_suffixes (list):** List of timepoint suffixes.
- **sigma (float):** Sigma value for the registration.
- **flip (bool):** Whether to flip the moving image based on metadata. (only works on images acquired with an Olympus system, set to false if not sure if images were acquired on an Olympus microscope)
- **dic (dict):** A dictionary of the image files with a reference image as the key and a list of moving images as the value.
- **skip_existing (bool):** Whether to skip already registered images.


In [ ]:
# Create an instance of the ImageRegistration class
registration = ImageRegistration(images = images, 
                                 out_directory = params['out_directory'], 
                                 in_filename_extension = params['in_filename_extension'], 
                                 final_filename_extension = params['final_filename_extension'], 
                                 timepoint_suffixes = params['timepoint_suffixes'], 
                                 sigma = params['sigma'], 
                                 flip = params['flip'], 
                                 dic = params['dic'],
                                 skip_existing=True)

In [ ]:
# Call the register_images method
registration.register_images()

## Step 2: Predict segmentation results

### !!! Faster With GPUs !!!

Requires GPUs with 40 GB vRAM otherwise disable GPUS

In [14]:
from os.path import exists
from os import mkdir
from pathlib import Path
from re import sub
import warnings
from numpy.random import shuffle
from novas3d.predict import PredictWarped

In [19]:
# Define the path to the mouse data
image_path = Path('novas3d_example_data/raw_warped')
images = list(image_path.glob('*res*.tif'))
images = sorted([x.as_posix() for x in images])
shuffle(images)

# Create data dictionaries for prediction
params = {
    "in_dir": "novas3d_example_data/raw_warped", # imput directory of files to segment
    "out_dir": "novas3d_example_data/raw_warped_seg", # output directory for files after segmentation
    "num_evals": 3, # number of evaluations for the ensemble to predict with
    "base_file_extension": ".tif", # base file extension
    "pred_file_extension": "_pred.npy", # predicted probabilities filename extension
    "mean_file_extension": "_mean.npy", # mean probabilities filename extension
}

if not exists(params['out_dir']):
    mkdir(params['out_dir'])

##################################################################################################################
# STD will be calcualted later on an allocation without GPUs, this takes up alot of CPU time leaving GPUs idol
##################################################################################################################

data_dict = [
    {"image": image_name}
    for image_name in images if not exists(sub(params["base_file_extension"], params["pred_file_extension"], sub(params["in_dir"], params["out_dir"], image_name)))
]

print(len(data_dict))

14


*class PredictWarped(images, out_directory, in_filename_extension, final_filename_extension, timepoint_suffixes, sigma, flip, dic, skip_finished=False)*

Class for Performing Prediction Using the Registered Images. Saves the predicted images.

**Args:**
- **data_dict (dict):** A dictionary containing the data for prediction.
- **config (dict):** A dictionary containing the configuration parameters.
    - **in_dir (str):** The input directory.
    - **out_dir (str):** The output directory.
    - **num_evals (int):** The number of evaluations.
    - **base_file_extension (str):** The base file extension.
    - **pred_file_extension (str):** The prediction file extension.
    - **mean_file_extension (str):** The mean file extension.
- **parameter_file (str):** The file path to the parameter file.
- **spatial_dims (int):** The number of spatial dimensions.
- **in_channels (int):** The number of input channels.
- **out_channels (int):** The number of output channels.
- **img_size (tuple):** The size of the input image.
- **feature_size (int):** The size of the features.
- **hidden_size (int):** The size of the hidden layers.
- **mlp_dim (int):** The dimension of the MLP layers.
- **pos_embed (bool):** Whether to use positional embedding.
- **res_block (bool):** Whether to use residual blocks.
- **norm_name (str):** The name of the normalization method.
- **spacing (tuple):** The spacing between voxels. Reciprocal of the voxel size in um to resample to 1 um.
- **i_min (float):** The minimum intensity value of raw images.
- **i_max (float):** The maximum intensity value of raw images.
- **b_min (float):** The minimum background value after normalization.
- **b_max (float):** The maximum background value after normalization.
- **clip (bool):** Whether to clip the values.
- **channel_dim (int):** The dimension index of the colour channel.
- **gpu (bool):** Whether to use the GPU.
- **skip_existing (bool):** Whether to skip images that have already been predicted.

**Methods:**
- **get_model():** Retrieves the model for prediction.
- **get_pred_transforms():** Retrieves the prediction transforms.
- **dataset_creation():** Creates the dataset for prediction.
- **prediction():** Performs the prediction.


In [20]:
PredictMatt = PredictWarped(data_dict, 
                            params,
                            parameter_file='NOVAS3D_Vessel_and_Neuron_Segmentation/parameters.pickle', 
                            spatial_dims=3, 
                            in_channels=2, 
                            out_channels=3, 
                            img_size=(128,128,128), 
                            feature_size=16, 
                            hidden_size=768, 
                            mlp_dim=3072, 
                            pos_embed="perceptron",
                            res_block=True, 
                            norm_name="instance",
                            spacing=(1, 1, 0.375), 
                            i_min=0, 
                            i_max=1024, 
                            b_min=0.0, 
                            b_max=1.0, 
                            clip=True, 
                            channel_dim=0,
                            gpu=False,
                            skip_existing=True)
PredictMatt.prediction()

  0%|                                                     | 0/3 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 7.78 GiB total capacity; 3.30 GiB already allocated; 697.25 MiB free; 4.02 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Step 3: Calculate STD of prediction

In [ ]:
from pathlib import Path
from re import sub
from os.path import exists
from os import mkdir
from numpy.random import shuffle
from novas3d.vessel_graph import SaveStdFile

In [ ]:
params = {
    "data_directory": "novas3d_example_data/raw_warped_seg", #location of output probabilities from prediction
    "in_directory": "novas3d_example_data/raw_warped_seg",
    "out_directory": "novas3d_example_data/raw_warped_seg",
    "init_tag": 'pred', # tag for the predicited probabilities array in step 2 "pred_file_extension"
    "final_tag": 'std', # tag to save the standard deviation of the predicted probabiliteis under
    "skip_existing": True, # skip files that have already been processed
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])

In [ ]:
# Define the files list
path = Path(params["data_directory"])
files = list(path.glob('*_' + params['init_tag'] + '.npy'))
files = sorted([x.as_posix() for x in files])
files = [x for x in files if not exists(sub(params['in_directory'], params['out_directory'], sub(params['init_tag'], params['final_tag'], x)))]
print(len(files))

# Shuffle the files list
shuffle(files)

*class SaveStdFile(files, in_directory, out_directory, in_tag, final_tag,skip_existing=False)*

Initialize SaveStdFile Instance

This section initializes an instance of the `SaveStdFile` class, calculates the standard deviation for predicted data, and saves the standard deviation files.

**Args:**
- **files (list):** List of file paths.
- **in_directory (str):** Input directory path.
- **out_directory (str):** Output directory path.
- **in_tag (str):** Initial tag.
- **final_tag (str):** Final tag.
- **skip_existing (bool):** Skip files where std already exists.

**Methods:**
- **std():** Save the standard deviation files.

In [ ]:
 # Create an instance of the SaveStdFile class with the files list and other parameters
save_std = SaveStdFile(files, params['in_directory'], params['out_directory'], params['init_tag'], params['final_tag'], params['skip_finished'])
# Save the standard deviation files
save_std.std()

## Step 4: Binarize Vascular predictions

In [ ]:
from pathlib import Path
from numpy.random import shuffle
from pathlib import Path
from os.path import exists
from os import mkdir
from novas3d.vessel_graph import Binarize

In [ ]:
directory = Path('novas3d_example_data/raw_warped_seg')
files  = directory.glob('*-*_mean.npy')
files = sorted([x.as_posix() for x in files])
print(len(files))

parameters = {
    'min_prob': 0.75, # minimum probability for binarization to classify a voxel as a vessel
    'max_var': 0.2, # maximum standard deviation for binarization to classify a voxel as a vessel
    'in_directory': "novas3d_example_data/raw_warped_seg", # input directory
    'out_directory': "novas3d_example_data/raw_warped_seg", # output directory
    'mean_tag': 'mean', # tag for mean probaility
    'std_tag': 'std', # tag for mean standard deviation of probability
    'seg_tag': 'seg' # output tag for segmentation
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])

shuffle(files)

*class Binarize(files, min_prob, max_var, in_directory, out_directory, mean_tag, std_tag, seg_tag, skip_existing=False)*
Initializes the `Binarize` class. This class is used to binarize the files in the specified directory based on mean and standard deviation thresholds.

**Args:**
- **directory (str):** The directory path.
- **min_prob (float):** The minimum probability threshold.
- **max_var (float):** The maximum variance threshold.
- **in_directory (str):** The input directory path.
- **out_directory (str):** The output directory path.
- **mean_tag (str):** The mean tag.
- **std_tag (str):** The standard deviation tag.
- **seg_tag (str):** The segmentation tag.
- **skip_existing (bool):** Skip existing files that have already been binarized.

**Methods:**
- **binarize_files():** Binarizes the files in the directory.
- **get_files():** Retrieves the list of files in the directory.


In [ ]:
binarizer = Binarize(files = files,
                     min_prob = parameters['min_prob'],
                     max_var = parameters['max_var'], 
                     in_directory = parameters['in_directory'],
                     out_directory = parameters['out_directory'], 
                     mean_tag = parameters['mean_tag'],
                     std_tag = parameters['std_tag'],
                     seg_tag = parameters['seg_tag'],
                     skip_existing=True)

binarizer.binarize_files()


## Step 4a: binarize neuron segmentation and calculate distances from every nonneuron pixel to the closest neuron

In [ ]:
import numpy as np
from pathlib import Path
from novas3d.vessel_graph import NeuronDistanceCalculator
from os.path import exists
from os import mkdir
from re import sub

In [ ]:
directory = Path('novas3d_example_data/raw_warped_seg')
files  = directory.glob('*-*_mean.npy')
files = sorted([x.as_posix() for x in files])
files = [x for x in files if not exists(sub('_mean.npy', '_seg_nrn.npy', x))]
print(len(files))

parameters = {
    'min_prob': 0.75,
    'max_var': 0.1,
    'in_directory': 'novas3d_example_data/raw_warped_seg',
    'out_directory': 'novas3d_example_data/raw_warped_seg',
    'mean_tag': 'mean',
    'std_tag': 'std',
    'neuron_distance_tag': 'seg_nrn_dst',
    'seg_tag': 'seg',
    'seg_nrn_tag': 'seg_nrn'   
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])

np.random.shuffle(files)

*class NeuronDistanceCalculator(files, min_prob, max_var, in_directory, out_directory, mean_tag, std_tag, neuron_distance_tag, seg_tag, seg_nrn_tag,skip_existing=False)*
Calculates the distance between neurons based on given parameters. Saves the distance transform of the Neuron segmentation array.

**Args:**
- **files (list):** List of file paths.
- **min_prob (float):** Minimum probability threshold.
- **max_var (float):** Maximum variance threshold.
- **in_directory (str):** Input directory path.
- **out_directory (str):** Output directory path.
- **mean_tag (str):** Tag for mean values.
- **std_tag (str):** Tag for standard deviation values.
- **neuron_distance_tag (str):** Tag for neuron distance values.
- **seg_tag (str):** Tag for segmentation values.
- **seg_nrn_tag (str):** Tag for segmented neuron values.
- **skip_existing (bool):** Whether to skip images that have already been predicted.

**Methods:**
- **calculate_distance():** Calculates the distance between neurons for each file in the given list.


In [ ]:
n = NeuronDistanceCalculator(files, 
                             parameters['min_prob'], 
                             parameters['max_var'], 
                             parameters['in_directory'], 
                             parameters['out_directory'], 
                             parameters['mean_tag'], 
                             parameters['std_tag'], 
                             parameters['neuron_distance_tag'], 
                             parameters['seg_tag'], 
                             parameters['seg_nrn_tag'],
                             skip_existing=True)
n.calculate_distance()

## Step 5: Calculate union of matched files

In [ ]:
from numpy.random import shuffle
from pathlib import Path
from re import sub
from os.path import exists
from os import mkdir
import pickle
from vessel_morphology.vessel_graph import SegmentationMaskUnion

In [ ]:
with open('novas3d_example_data/matched_stacks.pickle', 'rb') as handle:
    dic = pickle.load(handle)
    
directory_seg = Path('novas3d_example_data/raw_warped_seg')
images = list(directory_seg.glob('*_0001_warped_seg.npy'))
images = sorted([x.as_posix() for x in images])
images = [x for x in images if exists(sub('_0001','',x))]
images = [x for x in images if any(y in x for y in list(dic.keys()))]

params = {'IOU_thresh':0.2,
        'input_directory':'novas3d_example_data/raw_warped_seg',
        'output_directory':'novas3d_example_data/raw_warped_seg',
        'initial_suffix':'_warped_seg.npy',
        'final_suffix_mat':'_seg_warped_single.mat',
        'final_suffix_npy':'_seg_warped_single.npy',
        'timepoint_suffixes':['_0001'],
        'thresh_remove_small_comps':250,
        'thresh_fill_holes':1000,
        'n_interations_binary_closing':3,
        }

if not exists(params['output_directory']):
    mkdir(params['output_directory'])

shuffle(images)
print(len(images))

*class SegmentationMaskUnion(files, dic, in_directory, out_directory, in_suffix, final_suffix_mat, final_suffix_npy, timepoint_suffixes, IOU_thresh, thresh_fill_holes, thresh_remove_small_comps, n_interations_binary_closing,skip_existing=False)*

Process a list of images by performing binary dilation, calculating intersection over union (IOU), performing post-processing, and saving the processed images as .mat and .npy files.

**Args:**
- **files (list):** List of image file paths.
- **dic (dict):** Dictionary containing additional information.
- **in_directory (str):** Path to the input directory.
- **out_directory (str):** Path to the output directory.
- **in_suffix (str):** Suffix of the initial files.
- **final_suffix_mat (str):** Suffix of the final .mat files.
- **final_suffix_npy (str):** Suffix of the final .npy files.
- **timepoint_suffixes (list):** List of timepoint suffixes. ex: ['_0001', '_0002', '_0003']
- **IOU_thresh (float):** Intersection over Union (IOU) threshold.
- **thresh_fill_holes (int):** Threshold for filling holes in the segmented image.
- **thresh_remove_small_comps (int):** Threshold for removing small connected components in the segmented image.
- **n_interations_binary_closing (int):** Number of iterations for binary closing.
- **skip_existing (bool):** Skip existing files that have already been processed.


In [ ]:
# Create an instance of ImageProcessor and process the images
processor = SegmentationMaskUnion(images, 
                                  dic,
                                  params['input_directory'], 
                                  params['output_directory'], 
                                  params['initial_suffix'], 
                                  params['final_suffix_mat'], 
                                  params['final_suffix_npy'],
                                  params['timepoint_suffixes'], 
                                  params['IOU_thresh'], 
                                  params['thresh_fill_holes'], 
                                  params['thresh_remove_small_comps'],
                                  params['n_interations_binary_closing'],
                                  skip_existing=True
                                  )
                                  
processor.process_images()

## Step 6: Generate Skeletons from the Union of the segmentation masks

Run the following code in matlab after modifying the following matlab file gen_skeletons_warped_single.m

module load matlab 


matlab -nodisplay -nodesktop -r "gen_skeletons_warped_single"

## Step 7: Convert skeletons into graphs

In [ ]:
from numpy.random import shuffle
from pathlib import Path
import pickle
from novas3d.vessel_graph import GraphGenerator

In [ ]:
with open('novas3d_example_data/matched_stacks.pickle', 'rb') as handle:
        dic = pickle.load(handle)
        
directory = Path('novas3d_example_data/raw_warped_seg')
files_seg_0001 = directory.glob('*_0001_warped_seg.npy')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
files_seg_0001 = [x for x in files_seg_0001 if  any(y in x for y in list(dic.keys()))]
shuffle(files_seg_0001)


params = {'IOU_thresh':0.2,
        'initial_suffix':'_warped_seg.npy',
        'initial_suffix_mat':'_skel_warped_single.mat',
        'final_suffix_npy':'_seg_warped_single.npy',
        'final_suffix_pickle':'_warped.pickle',
        'final_suffix_tif':'_single_skel.tif',
        'timepoint_suffixes':['_0001'],
        'ref_timepoint':'_0001',
        'thresh_remove_terminal_segemnts':5,
        'input_directory':'raw_warped_single_upsampled_seg',
        'output_directory':'raw_warped_single_upsampled_seg',
        }

*class GraphGenerator(files, dic, in_directory, out_directory, in_suffix, final_suffix_pickle, in_suffix_mat, final_suffix_tif, ref_timepoint, timepoint_suffixes, IOU_thresh, thresh_remove_terminal_segemnts, skip_existing=False)*

Generates graphs based on input files and parameters. Saves the graphs as pickle files.

**Args:**
- **files (list):** List of input file paths.
- **dic (dict):** Dictionary containing paired files. Key is the reference file and value is a list of matching files.
- **in_directory (str):** Path to the input directory.
- **out_directory (str):** Path to the output directory.
- **in_suffix (str):** Suffix of the initial files.
- **final_suffix_pickle (str):** Suffix of the final pickle files.
- **final_suffix_mat (str):** Suffix of the final mat files.
- **final_suffix_tif (str):** Suffix of the final tif files.
- **ref_timepoint (str):** Reference timepoint.
- **timepoint_suffixes (list):** List of timepoint suffixes.
- **IOU_thresh (float):** IOU threshold.
- **thresh_remove_terminal_segments (int):** Threshold to remove terminal segments.
- **skip_existing (bool):** Skip existing files that have already been processed.

**Methods:**
- **process_files():** Process the input files and generate graphs.


In [ ]:
MakeGraphs = GraphGenerator(files = files_seg_0001,
                            dic = dic, 
                            in_directory=params['input_directory'], 
                            out_directory = params['output_directory'], 
                            in_suffix = params['initial_suffix'], 
                            final_suffix_pickle = params['final_suffix_pickle'], 
                            in_suffix_mat = params['initial_suffix_mat'], 
                            final_suffix_tif = params['final_suffix_tif'], 
                            ref_timepoint = params['ref_timepoint'], 
                            timepoint_suffixes = params['timepoint_suffixes'], 
                            IOU_thresh = params['IOU_thresh'], 
                            thresh_remove_terminal_segemnts = params['thresh_remove_terminal_segemnts'],
                            skip_existing=True)

MakeGraphs.generate_graphs()

## Step 8: Calculate radii for vessel segments

In [ ]:
from numpy import array
from numpy.random import shuffle
from pathlib import Path
from re import sub
from os.path import exists
from os import mkdir
from novas3d.vessel_graph import VesselRadiiCalc

In [ ]:
params = {
    'in_directory': 'novas3d_example_data/raw_warped_seg',
    'img_directory': 'novas3d_example_data/raw_warped',
    'mean_directory': 'novas3d_example_data/raw_warped_seg',
    'std_directory': 'novas3d_example_data/raw_warped_seg',
    'out_directory': 'novas3d_example_data/graphs',
    'pickle_file_suffix': '_warped.pickle',
    'out_pickle_suffix': '_radii.pickle',
    'img_suffix': '_warped.tif',
    'seg_suffix': '_warped_seg.npy',
    'mean_suffix': '_warped_mean.npy',
    'std_suffix': '_warped_std.npy',
    'neuron_distance_suffix': '_warped_seg_nrn_dst.npy',
    'second_channel': True,
    'neuron_channel': True,
    'psf': array([0.636,0.127,0.127]),
    'spacing': (1,1,2.645833333),
    'vessel_segment_limit': 2000,
    'max_pixel_value': 1023,
    'n_iter_deconv': 10,
    'grid_size_psf_deconv': 31, # must be odd number
    'sampling': 1/5,
    'n_cores':16
}

if not exists(params['out_directory']):
    mkdir(params['out_directory'])

directory = Path(params['in_directory'])
files = directory.glob('*' + params['pickle_file_suffix'])
files = sorted([x.as_posix() for x in files])
files = [x for x in files if '-' in x]
files = [x for x in files if not exists(sub(params['in_directory'],params['out_directory'],sub(params['pickle_file_suffix'],params['out_pickle_suffix'],x)))]
print(len(files))
shuffle(files)

*class VesselRadiiCalc(files, in_directory, img_directory, mean_directory, std_directory, out_directory, pickle_file_suffix, out_pickle_suffix, img_suffix, seg_suffix, mean_suffix, std_suffix, neuron_distance_suffix, second_channel, neuron_channel, psf, spacing, vessel_segment_limit, max_pixel_value, n_iter_deconv, grid_size_psf_deconv, sampling, n_cores, filter_radii = False, butter_N = None, butter_Wn = None, butter_btype = None, butter_fs = None, skip_existing=False)*
Initializes an instance of the `VesselRadiiCalc` class. Calculates the radii of vessels based on given parameters and saves the radii in the graph.

**Args:**
- **files (list):** List of file paths.
- **in_directory (str):** Input directory path.
- **img_directory (str):** Image directory path.
- **mean_directory (str):** Mean directory path.
- **std_directory (str):** Standard deviation directory path.
- **out_directory (str):** Output directory path.
- **pickle_file_suffix (str):** Suffix of the pickle files.
- **out_pickle_suffix (str):** Suffix of the output pickle files.
- **img_suffix (str):** Suffix of the image files.
- **seg_suffix (str):** Suffix of the segmented files.
- **mean_suffix (str):** Suffix of the mean files.
- **std_suffix (str):** Suffix of the standard deviation files.
- **neuron_distance_suffix (str):** Suffix of the neuron distance files.
- **second_channel (bool):** Second channel flag. Denotes the presence of a second channel.
- **neuron_channel (bool):** Neuron channel flag. Denotes the presence of a neuron channel.
- **psf (list):** Point spread function.
- **spacing (float):** Spacing value for pixel size along each axis.
- **vessel_segment_limit (int):** Vessel segment limit. If larger than the limit, the file is skipped.
- **max_pixel_value (int):** Maximum pixel value.
- **n_iter_deconv (int):** Number of iterations for deconvolution.
- **grid_size_psf_deconv (int):** Grid size for deconvolution. This must be an odd number.
- **sampling (float):** Sampling in microns to calculate the radii.
- **n_cores (int):** Number of cores.
- **filter_radii (bool):** Filter radii with a Butterworth filter.
- **butter_N (int):** Butterworth filter order.
- **butter_Wn (float):** Butterworth filter frequency.
- **butter_btype (str):** Butterworth filter type.
- **butter_fs (float):** Butterworth filter frequency.
- **skip_existing (bool):** Skip existing files that have already been processed.

**Returns:**
- Saves the radii in the graph, along with the standard deviation of the radius estimates along the path.
- If the `filter_radii` flag is set to `True`, the radii are filtered with a Butterworth filter and saved under the `path_weights` key in the graph. The unfiltered radii are saved under the `path_weights_unfiltered` key in the graph.
- If the `filter_radii` flag is set to `False`, the radii are saved as is under the `path_weights` key in the graph.
- In both cases, the mean radii estimate is calculated from unfiltered radii estimates and saved under the `radii` key in the graph.
- The standard deviation of the radii estimates is calculated and saved under the `radii_std` key in the graph.
- If `neuron_channel` is set to `True`, the mean distance to the closest neuron is calculated and saved under the `mean_neuron_distance` key in the graph.
- If `neuron_channel` is set to `True`, the standard deviation of the distance to the closest neuron is calculated and saved under the `neuron_distance_std` key in the graph.
- If `neuron_channel` is set to `True`, the distance to the closest neuron is saved under the `neuron_distance_min` key in the graph.


In [ ]:
vesselRadiusEstimator = VesselRadiiCalc(files,
                                        params['in_directory'],
                                        params['img_directory'],
                                        params['mean_directory'],
                                        params['std_directory'],
                                        params['out_directory'],
                                        params['pickle_file_suffix'],
                                        params['out_pickle_suffix'],
                                        params['img_suffix'],
                                        params['seg_suffix'],
                                        params['mean_suffix'],
                                        params['std_suffix'],
                                        params['neuron_distance_suffix'],
                                        params['second_channel'],
                                        params['neuron_channel'],
                                        params['psf'],
                                        params['spacing'],
                                        params['vessel_segment_limit'],
                                        params['max_pixel_value'],
                                        params['n_iter_deconv'],
                                        params['grid_size_psf_deconv'],
                                        params['sampling'],
                                        params['n_cores'],
                                        skip_exsiting=True)
vesselRadiusEstimator.process_all_files()